In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.stats as stats

In [3]:
trajectory_df = pd.read_excel("2014_BFncomms_suppledata1.xlsx",skiprows=4)
print(trajectory_df.shape)
trajectory_df[:2]

(4014, 10)


,Code,Text,Code.1,Text.1,N patients,RR,estimated,sampled,N samples,p-value
0,C34,Malignant neoplasm of bronchus and lung,D63,Anaemia in chronic diseases classified elsewhere,2889,3.097193,0.0,< 1.214E-08,82360600,0.0
1,C50,Malignant neoplasm of breast,C78,Secondary malignant neoplasm of respiratory an...,5890,3.812002,0.0,< 1.214E-08,82360600,0.0


In [4]:
sym_fea = pd.read_csv("../../../../extract_pubmed/2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# UMLS-ICD10
mapping_df = pd.read_csv("../DisGeNET-disease_mappings.tsv",sep="\t")
mapping_df = mapping_df[mapping_df["vocabulary"]=="ICD10"]
mapping_df.reset_index(drop=True,inplace=True)
print(mapping_df.shape)
mapping_df[:2]

(2788, 5)


,diseaseId,name,vocabulary,code,vocabularyName
0,C0008354,Cholera,ICD10,A00,Cholera
1,C0178238,Intestinal infectious disease (disorder),ICD10,A00-A09.9,Intestinal infectious diseases


### ICD10 trans MSH
- ICD10 --> UMLS -->MSH  ,288-->135-->85 diseases

In [6]:
#trajectory_df 
dis_in_trajector = set(trajectory_df["Text"]) & set(trajectory_df["Text.1"])
print(len(dis_in_trajector))
dis_in_trajector = list(dis_in_trajector)

288


In [7]:
# 1、ICD10->UMLS
icd_to_umls = {}
for ite in range(len(mapping_df)):
    vacName = mapping_df.loc[ite,"vocabularyName"]
    if vacName in dis_in_trajector:
        icd_to_umls[vacName] = mapping_df.loc[ite,"name"]
        
print(len(icd_to_umls))

135


In [8]:
# 2、UMLS->MESH
mapping_df_MSH = pd.read_csv("../DisGeNET-disease_mappings.tsv",sep="\t")
mapping_df_MSH = mapping_df_MSH[mapping_df_MSH["vocabulary"]=="MSH"]
mapping_df_MSH.reset_index(drop=True,inplace=True)
print(mapping_df_MSH.shape)
print(mapping_df_MSH[:2])

# UMLS 
umls_dis_ls = list(icd_to_umls.values())
print(len(umls_dis_ls))
print(len(set(umls_dis_ls))) 

(29335, 5)
  diseaseId                       name vocabulary        code  \
0  C1970109  AROMATASE EXCESS SYNDROME        MSH  C000591739   
1  C1970109  AROMATASE EXCESS SYNDROME        MSH  C000591739   

                                      vocabularyName  
0                          Aromatase Excess Syndrome  
1  Familial gynecomastia, due to increased aromat...  
135
135


In [9]:
# UMLS->MESH
umls_to_mesh ={}
for umls_dis in umls_dis_ls:
    umls_toMSH_df = mapping_df_MSH[mapping_df_MSH["name"]==umls_dis]
    if len(umls_dis_ls)>0:
        print(len(umls_toMSH_df))
        vacName_df = umls_toMSH_df["vocabularyName"]
        for dis_MSH in list(vacName_df):
            if dis_MSH in list(sym_fea.index):
                umls_to_mesh[umls_dis] = dis_MSH
                
print(len(umls_to_mesh)) #85

1
1
0
1
1
0
2
2
0
2
2
3
2
3
3
2
1
0
5
0
0
3
2
3
0
3
7
0
0
3
7
1
3
2
3
3
1
4
6
2
1
1
0
3
1
4
0
0
4
0
5
1
16
2
4
7
0
2
3
9
0
0
1
1
0
1
0
1
0
0
2
3
6
5
1
6
3
2
0
0
0
1
3
1
0
0
0
1
1
1
1
3
1
2
2
2
0
1
0
2
2
2
1
3
2
0
1
1
2
2
1
1
5
1
2
1
2
4
0
1
0
0
0
1
2
0
0
1
0
0
2
4
1
0
5
85


In [10]:
umls_to_mesh_df = pd.DataFrame(umls_to_mesh,index=["dis_mesh"]).T
umls_to_mesh_df.reset_index(inplace=True)
umls_to_mesh_df.rename(columns={"index":"dis_umls"},inplace=True)
print(umls_to_mesh_df.shape)
umls_to_mesh_df[:2]

(85, 2)


,dis_umls,dis_mesh
0,Erysipelas,Erysipelas
1,Herpes zoster disease,Herpes Zoster


In [11]:
# 3、ICD10->MESH 
icd_to_mesh={}

for dis_icd in icd_to_umls.keys():
    dis_umls = icd_to_umls[dis_icd]
    if umls_to_mesh.__contains__(dis_umls):
        icd_to_mesh[dis_icd]=umls_to_mesh[dis_umls]
        
print("ICD10:MESH 映射的 关系数：",len(icd_to_mesh))

ICD10:MESH 映射的 关系数： 85


### Extract trajectory  for 85 diseases
- 152

In [12]:
trajectory_trans = []

for ite in range(len(trajectory_df)):
    disA = trajectory_df.loc[ite,"Text"]
    disB = trajectory_df.loc[ite,"Text.1"]
    
    if (disA in icd_to_mesh.keys()) and (disB in icd_to_mesh.keys()):
        RR = trajectory_df.loc[ite,"RR"]
        trajectory_trans.append([icd_to_mesh[disA],icd_to_mesh[disB],RR])

trajectory_trans_df = pd.DataFrame(trajectory_trans)
trajectory_trans_df.columns=["disA","disB","RR"]
print(trajectory_trans_df.shape)
trajectory_trans_df[:2]

(152, 3)


,disA,disB,RR
0,"Diabetes Mellitus, Type 2","Diabetes Mellitus, Type 1",6.086008
1,"Diabetes Mellitus, Type 2","Kidney Failure, Chronic",2.442088


In [13]:
# trajectory_trans_df.to_csv("trajectory_after_idtrans.csv")

### Compare the differences
- Pair with 85 diseases，7140/2 =3570 pairs（Because of the directionality, AB/BA counts as two pairs, but the predicted value counts as one pair. So sum the RR of AB/BA as the RR of a pair）

In [73]:
predict_score = pd.read_csv("../../visualization/model_predictAll_result.csv",index_col=0)

#extract index of 85 diseases in 4268
dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in tqdm(icd_to_mesh.values()):
    index_inSym = sym_fea_index_lower.index(dis.lower())
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:2]

E:\Anaconda\envs\tf2\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
100%|██████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<?, ?it/s]

85


[3333, 2113]

In [99]:
scoreABs = []
scoreBAs = []
trajectory_confs = [] 
trajectory_ABs = []
trajectory_BAs = []

for indexA in tqdm(range(len(icd_to_mesh.values()))):
    disA = list(icd_to_mesh.values())[indexA]
    disA_index = dis_index_inSym[list(icd_to_mesh.values()).index(disA)]
    
    for indexB in range(indexA+1,len(icd_to_mesh.values())):
        disB = list(icd_to_mesh.values())[indexB]
        disB_index = dis_index_inSym[list(icd_to_mesh.values()).index(disB)]
        
        scoreAB = predict_score.loc[disA_index*4268+disB_index]
        scoreBA = predict_score.loc[disB_index*4268+disA_index]
        scoreABs.append(scoreAB)
        scoreBAs.append(scoreBA)
            
        #construct RR
        con_df_AB = trajectory_trans_df.loc[(trajectory_trans_df["disA"]==disA) & (trajectory_trans_df["disB"]==disB)]
        if len(con_df_AB)>0:
            RR = list(trajectory_trans_df["RR"])[0]
            trajectory_ABs.append(RR)
        else:
            trajectory_ABs.append(0.0)
            
        con_df_BA = trajectory_trans_df.loc[(trajectory_trans_df["disA"]==disB) & (trajectory_trans_df["disB"]==disA)]
        if len(con_df_BA)>0:
            RR = list(trajectory_trans_df["RR"])[0]
            trajectory_BAs.append(RR)
        else:
            trajectory_BAs.append(0.0)
            
print(len(trajectory_ABs),len(trajectory_ABs)==len(scoreABs))
trajectory_ABs[:2]

100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [00:05<00:00, 15.71it/s]

3570 True


[0.0, 0.0]

In [100]:
# average scoreAB and scoreBA as similarity of disease A and disease B
scores_predict=[] 
scores_predict = np.sum([scoreABs,scoreBAs],axis=0)/2
scores_predict = scores_predict.squeeze()

print(scores_predict[:10])

trajectory_confs = np.sum([trajectory_ABs,trajectory_BAs],axis=0)/2
trajectory_confs = trajectory_confs.squeeze()

print(trajectory_confs[:10])

[9.83330625e-01 2.79728812e-01 9.82884150e-01 2.88282790e-01
 3.88150999e-01 3.59595419e-01 2.54322404e-03 2.54322404e-03
 1.67670472e-03 6.02645690e-04]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [101]:
trajectory_sim = []
trajectory_unsim = []

for i in range(len(scores_predict)):
    score = scores_predict[i]
    if score>=0.5:
        trajectory_sim.append(trajectory_confs[i])
    elif score<0.5:
        trajectory_unsim.append(trajectory_confs[i])
        
print(len(trajectory_sim),"--",len(trajectory_unsim))

1356 -- 2214


#### Calculate distribution difference

In [103]:
distribution_dif = stats.mannwhitneyu(trajectory_sim,trajectory_unsim,alternative='two-sided')
distribution_dif

MannwhitneyuResult(statistic=1517631.0, pvalue=0.11359433360074186)

In [104]:
print(np.mean(trajectory_sim))
print(np.mean(trajectory_unsim))

0.1503549174041298
0.11682716350496837


In [105]:
# pd.Series(trajectory_sim).to_csv("trajectory_sim.csv",index="RR")
# pd.Series(trajectory_unsim).to_csv("trajectory_unsim.csv",index="RR")